In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

### Features Detection

In [2]:
sg_tsv = './features_detection/training/test.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})
df_feature_detection

,id,features
0,1007129816,{'features': 'AAAAAAAAAAAAAAAAAAAAAPXb+T5GtMtA...
1,1009434119,{'features': 'MGCwP7rIGD8AAAAADmf2P/t0MzwAAAAA...
2,101362133,{'features': 'xtUpPdry1j70gqs9uakFP0iUaT8AAAAA...
3,102617084,{'features': 'eiPzPuvL90AAAAAAMWdqPgrM7kAAAAAA...
4,10287332,{'features': 'PYSNPnWZ0z44gPc+AAAAAAAAAAAAAAAA...
...,...,...
995,900144365,{'features': 'zmRlPgAAAAAAAAAAAAAAAA38KUAAAAAA...
996,94024624,{'features': 'D1PgPQAAAAAAAAAAAAAAAC46jUAAAAAA...
997,95758790,{'features': 'Kfx7P5dZ0DwAAAAAmWwaQAAAAAAAAAAA...
998,97233789,{'features': '61RLPyLOaD4z3JY+AAAAAAAAAAAAAAAA...


In [3]:
sg_tsv = './features_detection/training/test2.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
df_label_detection

,id,label
0,1007129816,"[{'class': 'Ear', 'conf': 0.970589816570282, '..."
1,1009434119,"[{'class': 'Dog', 'conf': 0.9937595725059509, ..."
2,101362133,"[{'class': 'Pant', 'conf': 0.8839820623397827,..."
3,102617084,"[{'class': 'Helmet', 'conf': 0.903003513813018..."
4,10287332,"[{'class': 'Man', 'conf': 0.9057866334915161, ..."
...,...,...
995,900144365,"[{'class': 'Short', 'conf': 0.9517521262168884..."
996,94024624,"[{'class': 'Hat', 'conf': 0.9739574790000916, ..."
997,95758790,"[{'class': 'Tree', 'conf': 0.9424041509628296,..."
998,97233789,"[{'class': 'Man', 'conf': 0.9472337365150452, ..."


In [4]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1007129816,{'features': 'AAAAAAAAAAAAAAAAAAAAAPXb+T5GtMtA...,"[{'class': 'Ear', 'conf': 0.970589816570282, '..."
1,1009434119,{'features': 'MGCwP7rIGD8AAAAADmf2P/t0MzwAAAAA...,"[{'class': 'Dog', 'conf': 0.9937595725059509, ..."
2,101362133,{'features': 'xtUpPdry1j70gqs9uakFP0iUaT8AAAAA...,"[{'class': 'Pant', 'conf': 0.8839820623397827,..."
3,102617084,{'features': 'eiPzPuvL90AAAAAAMWdqPgrM7kAAAAAA...,"[{'class': 'Helmet', 'conf': 0.903003513813018..."
4,10287332,{'features': 'PYSNPnWZ0z44gPc+AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9057866334915161, ..."
...,...,...,...
995,900144365,{'features': 'zmRlPgAAAAAAAAAAAAAAAA38KUAAAAAA...,"[{'class': 'Short', 'conf': 0.9517521262168884..."
996,94024624,{'features': 'D1PgPQAAAAAAAAAAAAAAAC46jUAAAAAA...,"[{'class': 'Hat', 'conf': 0.9739574790000916, ..."
997,95758790,{'features': 'Kfx7P5dZ0DwAAAAAmWwaQAAAAAAAAAAA...,"[{'class': 'Tree', 'conf': 0.9424041509628296,..."
998,97233789,{'features': '61RLPyLOaD4z3JY+AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9472337365150452, ..."


In [5]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:05, 187.21it/s]


### Features Segmentation

In [6]:
sg_tsv = './features_test1/training/test.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})
df_feature_segmentation

,id,features
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...
...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...


In [7]:
sg_tsv = './features_test1/training/test.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
df_label_segmentation

,id,label
0,1007129816,"[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,"[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,"[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,"[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,"[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...
995,900144365,"[{'class': 'Tennis racket', 'conf': 0.73602569..."
996,94024624,"[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,"[{'class': 'Skirt', 'conf': 0.9404543042182922..."
998,97233789,"[{'class': 'Ring', 'conf': 0.5740575194358826,..."


In [8]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...,"[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...,"[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...,"[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...,"[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...,"[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...,"[{'class': 'Tennis racket', 'conf': 0.73602569..."
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...,"[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...,"[{'class': 'Skirt', 'conf': 0.9404543042182922..."
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...,"[{'class': 'Ring', 'conf': 0.5740575194358826,..."


In [9]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:01, 508.29it/s]


In [10]:
import en_core_web_md
nlp = en_core_web_md.load()

Metodo 1: effettua una similarità sulla bounding box e conta il numero di volte con cui una classe si ripete, alla fine prende quella con la frequenza maggiore. <br>
Metodo 2: prende la classe con bounding box più simile e se ci sono classi candidate prende quella con la confidenza di predizione maggiore. <br>
Metodo 3: prende la classe con le features della regione più simili e se ci sono classi candidate prende quella con la confidenza di predizione maggiore. <br>

Su ogni metodo viene calcolato uno score pesato che considera la similarità delle features, del word embeddings e la confidenza di predizione.
La somma dei pesi applicati alle componenti dello score è 1, distribuzione:
- peso_feat = 0.5

- peso_word = 0.3

- peso_conf = 0.2

score = sim_feat * peso_feat + sim_word * peso_word + conf * peso_conf

Viene presa la classe del metodo che ottiene lo score più alto

In [11]:
#full_segmentation[full_segmentation["id"] == 1007129816]

In [12]:
bbox_similarity = 0.997
#index = 250 # ?
#index = 103
#index = 118
#index = 851
#index = 38
#index = 591 # ?

#for elem in full_segmentation.iloc[index]["label"]:
for index, row in tqdm(full_segmentation.iterrows()):
    for elem in row["label"]:
        similar_region = {"class": "", "score": 0, "feature": [], "conf": 0}
        similar_regions = {}
        similar_regions_features = {}
        similar_features = {"class": "", "score": 0, "feature": [], "conf": 0}
        for elem2 in full_detection.iloc[index]["label"]:

            if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > bbox_similarity:
                #print("similar region class:", elem2["class"])
                similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
                sim_features = 1 - spatial.distance.cosine(elem["feature"], elem2["feature"])
                if (sim_features > similar_features["score"]): # 3
                    similar_features["score"] = sim_features
                    similar_features["class"] = elem2["class"]
                    similar_features["feature"] = elem2["feature"]
                    similar_features["conf"] = elem2["conf"]
                if (similarity > similar_region["score"]): # 1
                    similar_region["score"] = similarity
                    similar_region["class"] = elem2["class"]
                    similar_region["feature"] = elem2["feature"]
                    similar_region["conf"] = elem2["conf"]
                if elem2["class"] in similar_regions: # 2
                    similar_regions[elem2["class"]] += 1
                    if similarity > similar_regions_features[elem2["class"]]["score"]: 
                        similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
                else:
                    similar_regions[elem2["class"]] = 0
                    similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
        if similar_region["class"] != "":
            #print("\n======================================================================")
            final_class = elem["class"]
            elem["class"] = elem["class"].lower()
            #print("Classe vecchia: ", elem["class"])
            classe = max(similar_regions, key=similar_regions.get)
            #print("Classe nuova metodo 1: ", classe)
            sim1 = 1 - spatial.distance.cosine(elem["feature"], similar_regions_features[classe]["feature"])
            tokens = nlp(elem["class"].lower() + " " + classe.lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_1 = token1.similarity(token2)
            #print("Classe nuova metodo 2: ", similar_region["class"])
            sim2 = 1 - spatial.distance.cosine(elem["feature"], similar_region["feature"])
            tokens = nlp(elem["class"].lower() + " " + similar_region["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_2 = token1.similarity(token2)
            #print("Classe nuova metodo 3: ", similar_features["class"])
            tokens = nlp(elem["class"].lower() + " " + similar_features["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_3 = token1.similarity(token2)

            peso_feat = 0.6
            peso_word = 0.2
            peso_conf = 0.2

            score_1_sim_feat = similarity_seg_1
            score_1_sim_word = sim1
            score_1_conf = similar_regions_features[classe]["conf"]
            score_1_tot = (score_1_sim_feat*peso_feat + score_1_sim_word*peso_word + score_1_conf*peso_conf)

            score_2_sim_feat = similarity_seg_2
            score_2_sim_word = sim2
            score_2_conf = similar_region["conf"]
            score_2_tot = (score_2_sim_feat*peso_feat + score_2_sim_word*peso_word + score_2_conf*peso_conf)

            score_3_sim_feat = similarity_seg_3
            score_3_sim_word = similar_features["score"]
            score_3_conf = similar_features["conf"]
            score_3_tot = (score_3_sim_feat*peso_feat + score_3_sim_word*peso_word + score_3_conf*peso_conf)

            
            if score_1_tot >= score_2_tot and score_1_tot >= score_3_tot:
                final_class = classe
            elif score_2_tot >= score_1_tot and score_2_tot >= score_3_tot:
                final_class = similar_region["class"]
            elif score_2_tot >= score_1_tot and score_3_tot >= score_2_tot:
                final_class = similar_features["class"]
            elem["class"] = final_class
            #print("classe finale: ", elem["class"])
            #print("======================================================================\n")

7it [00:04,  1.19it/s]C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_7764/138208695.py:49: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_1 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_7764/138208695.py:54: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_2 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_7764/138208695.py:58: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_3 = token1.similarity(token2)
1000it [08:49,  1.89it/s]


In [ ]:
# full version (old)
'''import en_core_web_md
nlp = en_core_web_md.load()
index = 591
for elem in full_segmentation.iloc[index]["label"]:
    synonyms = []
    similar_region = {"class": "", "score": 0, "feature": [], "conf": 0}
    similar_regions = {}
    similar_regions_features = {}
    similar_features = {"class": "", "score": 0, "feature": [], "conf": 0}
    for syn in wordnet.synsets(elem["class"]):
        for l in syn.lemmas():
            synonyms.append(l.name().lower())
    for elem2 in full_detection.iloc[index]["label"]:
        if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > 0.9:
            #print("class segmentation: ", elem["class"])
            #print("class detection: ", elem2["class"])
            similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
            #print("similarity: ", similarity)
            sim_features = 1 - spatial.distance.cosine(elem["feature"], elem2["feature"])
            #print("similarity features: ", 1 - spatial.distance.cosine(elem["feature"], elem2["feature"]))
            if (sim_features > similar_features["score"]): # 3
                similar_features["score"] = sim_features
                similar_features["class"] = elem2["class"]
                similar_features["feature"] = elem2["feature"]
                similar_features["conf"] = elem2["conf"]
            if (similarity > similar_region["score"]): # 1
                similar_region["score"] = similarity
                similar_region["class"] = elem2["class"]
                similar_region["feature"] = elem2["feature"]
                similar_region["conf"] = elem2["conf"]
            if elem2["class"] in similar_regions: # 2
                similar_regions[elem2["class"]] += 1
                if similarity > similar_regions_features[elem2["class"]]["score"]: 
                    similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
            else:
                similar_regions[elem2["class"]] = 0
                similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
            #print("______________________________________________________________________")
    if similar_region["class"] != "":
        print("\n======================================================================")
        #print("sinonimi :", synonyms)
        elem["class"] = elem["class"].lower()
        print("Classe vecchia: ", elem["class"])
        classe = max(similar_regions, key=similar_regions.get)
        print("Classe nuova metodo 1: ", classe)
        sim1 = 1 - spatial.distance.cosine(elem["feature"], similar_regions_features[classe]["feature"])
        #print("Similarity fetures 1: ", sim1)
        #print("Conf 1: ", similar_regions_features[classe]["conf"])
        tokens = nlp(elem["class"].lower() + " " + classe.lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_1 = token1.similarity(token2)* 1.2
        #print("Similarity words 2: ", similarity_seg_1)
        print("Classe nuova metodo 2: ", similar_region["class"])
        sim2 = 1 - spatial.distance.cosine(elem["feature"], similar_region["feature"])
        #print("Similarity fetures 2: ", sim2)
        #print("Conf 2: ", similar_region["conf"])
        tokens = nlp(elem["class"].lower() + " " + similar_region["class"].lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_2 = token1.similarity(token2)* 1.2
        #print("Similarity words 2: ", similarity_seg_2)
        print("Classe nuova metodo 3: ", similar_features["class"])
        #print("Similarity fetures 3: ", similar_features["score"])
        #print("Conf 3: ", similar_features["conf"])
        tokens = nlp(elem["class"].lower() + " " + similar_features["class"].lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_3 = token1.similarity(token2)* 1.2
        #print("Similarity words 3: ", similarity_seg_3)
        
        score_1_sim_feat = similarity_seg_1
        score_1_sim_word = sim1
        score_1_conf = similar_regions_features[classe]["conf"]/5
        score_1_tot = (score_1_sim_feat + score_1_sim_word + score_1_conf)
        
        score_2_sim_feat = similarity_seg_2
        score_2_sim_word = sim2
        score_2_conf = similar_region["conf"]/5
        score_2_tot = (score_2_sim_feat + score_2_sim_word + score_2_conf)
        
        score_3_sim_feat = similarity_seg_3
        score_3_sim_word = similar_features["score"]
        score_3_conf = similar_features["conf"]/5
        score_3_tot = (score_3_sim_feat + score_3_sim_word + score_3_conf)
        
        print("111111111111111111111111111111111111111111111111111111111111")
        print("score 1:\n", score_1_tot)
        print("sim feat: ",score_1_sim_feat)
        print("sim word: ",score_1_sim_word)
        print("conf: ",score_1_conf)
        print("22222222222222222222222222222222222222222222222222222222222")
        print("score 2:\n", score_2_tot)
        print("sim feat: ",score_2_sim_feat)
        print("sim word: ",score_2_sim_word)
        print("conf: ",score_2_conf)
        print("333333333333333333333333333333333333333333333333333333333333")
        print("score 3:\n", score_3_tot)
        print("sim feat: ",score_3_sim_feat)
        print("sim word: ",score_3_sim_word)
        print("conf: ",score_3_conf)
        print("------------------------------------------------------------")
        if classe.lower() in elem["class"].split(" "):
            print("classe finale la 1 corrispondenza esatta: ", classe)
        elif similar_region["class"].lower() in elem["class"].split(" "):
            print("classe finale la 2 corrispondenza esatta: ", similar_region["class"])
        elif similar_features["class"].lower() in elem["class"].split(" "):
            print("classe finale la 3 corrispondenza esatta: ", similar_features["class"])
        elif classe.lower() in synonyms:
            print("classe finale la 1 corrispondenza sinonimi: ", classe)
        elif similar_region["class"].lower() in synonyms:
            print("classe finale la 2 corrispondenza sinonimi: ", similar_region["class"])
        elif similar_features["class"].lower() in synonyms:
            print("classe finale la 3 corrispondenza sinonimi: ", similar_features["class"])
        elif score_1_tot >= score_2_tot and score_1_tot >= score_3_tot:
            print("classe finale la 1 word similarity: ", classe)
        elif score_2_tot >= score_1_tot and score_2_tot >= score_3_tot:
            print("classe finale la 2 word similarity: ", similar_region["class"])
        elif score_2_tot >= score_1_tot and score_3_tot >= score_2_tot:
            print("classe finale la 3 word similarity: ", similar_features["class"])
        print("======================================================================\n")
        
        img = str(full_detection.iloc[index]["id"])+".jpg"
        im_cv2 = cv2.imread("../datasets/flickr30k_images/test/"+img)
        x,y,w,h = tuple(elem["rect"])
        ROI = im_cv2[int(y):int(h), int(x):int(w)]
        #print(str(elem["class"]))
        #print("area: ", ROI.shape[0]*ROI.shape[1])
        cv2.imshow(str(elem["class"]),ROI)
        cv2.waitKey(0) 
        cv2.destroyAllWindows() '''

### Union

In [15]:
full_not_present = []
for i in tqdm(range(len(full_detection))):    
    not_present = []
    for row_dect in full_detection.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation.iloc[i]["label"]:
            # take only not present region and add it into segmentation features
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985: 
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:53<00:00, 18.54it/s]


In [16]:
for elem in full_detection["id"].values == full_segmentation["id"].values:
    if not elem: 
        print("noooo")

In [17]:
full_segmentation["not_present"] = full_not_present

In [18]:
full_segmentation["union"] = full_segmentation["label"] + full_segmentation["not_present"]
full_segmentation

,id,features,label,not_present,union
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...,"[{'class': 'Earring', 'conf': 0.97952914237976...","[{'class': 'Shirt', 'conf': 0.8929285407066345...","[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...,"[{'class': 'Dog', 'conf': 0.9183911085128784, ...","[{'class': 'Ear', 'conf': 0.8722665905952454, ...","[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...,"[{'class': 'Belt', 'conf': 0.6726886630058289,...","[{'class': 'Pant', 'conf': 0.8839820623397827,...","[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...,"[{'class': 'Goggles', 'conf': 0.94563788175582...","[{'class': 'Pant', 'conf': 0.5980353951454163,...","[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...,"[{'class': 'Jean', 'conf': 0.7828388214111328,...","[{'class': 'Sky', 'conf': 0.8375805616378784, ...","[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...,...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...,"[{'class': 'Hat', 'conf': 0.7360256910324097, ...","[{'class': 'Shirt', 'conf': 0.8718886375427246...","[{'class': 'Hat', 'conf': 0.7360256910324097, ..."
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...,"[{'class': 'Hat', 'conf': 0.7257902026176453, ...","[{'class': 'Building', 'conf': 0.5173280239105...","[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...,"[{'class': 'Shirt', 'conf': 0.9404543042182922...","[{'class': 'Tree', 'conf': 0.9424041509628296,...","[{'class': 'Shirt', 'conf': 0.9404543042182922..."
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...,"[{'class': 'Bracelet', 'conf': 0.5740575194358...","[{'class': 'Game', 'conf': 0.7647553086280823,...","[{'class': 'Bracelet', 'conf': 0.5740575194358..."


In [19]:
for index, row in tqdm(full_segmentation.iterrows()):
    if(len(row["label"]) + len(row["not_present"]) != len(row["union"])):
        print("!!!!!!!!!!!!!!!")

1000it [00:00, 3780.15it/s]


In [20]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [21]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [22]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3380.89it/s]


In [23]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_class/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'test.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'test.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

path to ./training_union_class/ created


In [24]:
tsv_writer(full_segmentation[["id", "feature_union"]].values.tolist(), FEATURE_FILE)

In [25]:
tsv_writer(full_segmentation[["id", "label_union"]].values.tolist(),LABEL_FILE)

In [26]:
sg_tsv = './training_union_class/test.feature.tsv'
df_feature = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature = df_feature.rename(columns={0: "id", 1: "features"})
df_feature

,id,features
0,1007129816,{'features': 'D+2rPwAAAAByKB1AAAAAACOOk0DcLitA...
1,1009434119,{'features': 'aGc8P/Mi9kBJpBFAlrPsQChxVUF4gRU/...
2,101362133,{'features': 'tFivQIUShUHoiMs/A8UmQd2MLkButQBB...
3,102617084,{'features': '7VLJP8X5eUEAAAAAldc0QcF6lUBuVblA...
4,10287332,{'features': 'XBGdQMwJeUCZbihBEsRYQQi8E0GhFac/...
...,...,...
995,900144365,{'features': 'G6MZQfE+2kBO/11AAAAAALLO6EAy3Qs/...
996,94024624,{'features': 'l8BAQWJRbEDyLptA5drSQG50rkC6hQFA...
997,95758790,{'features': 'kBFhQSA+90BQ7A5A+nPgQImNxz8AAAAA...
998,97233789,{'features': 'AAAAAAAAAAAAAAAAAAAAAAAAAADmi9c/...


In [27]:
sg_tsv = './training_union_class/test.label.tsv'
df_label = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label = df_label.rename(columns={0: "id", 1: "label"})
df_label

,id,label
0,1007129816,"[{'class': 'Earring', 'conf': 0.97952914237976..."
1,1009434119,"[{'class': 'Dog', 'conf': 0.9183911085128784, ..."
2,101362133,"[{'class': 'Belt', 'conf': 0.6726886630058289,..."
3,102617084,"[{'class': 'Goggles', 'conf': 0.94563788175582..."
4,10287332,"[{'class': 'Jean', 'conf': 0.7828388214111328,..."
...,...,...
995,900144365,"[{'class': 'Hat', 'conf': 0.7360256910324097, ..."
996,94024624,"[{'class': 'Hat', 'conf': 0.7257902026176453, ..."
997,95758790,"[{'class': 'Shirt', 'conf': 0.9404543042182922..."
998,97233789,"[{'class': 'Bracelet', 'conf': 0.5740575194358..."


In [28]:
sg_tsv = './training_union/test.hw.tsv'
df_hw = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_hw = df_hw.rename(columns={0: "id", 1: "dim"})
df_hw

,id,dim
0,1007129816,"[{'height': 461, 'width': 500}]"
1,1009434119,"[{'height': 333, 'width': 500}]"
2,101362133,"[{'height': 500, 'width': 347}]"
3,102617084,"[{'height': 375, 'width': 500}]"
4,10287332,"[{'height': 375, 'width': 500}]"
...,...,...
995,900144365,"[{'height': 385, 'width': 500}]"
996,94024624,"[{'height': 332, 'width': 500}]"
997,95758790,"[{'height': 333, 'width': 500}]"
998,97233789,"[{'height': 333, 'width': 500}]"


In [33]:
np.all(df_hw["id"].values == df_feature["id"].values)

True

In [32]:
np.all(df_hw["id"].values == df_label["id"].values)

True